In [12]:
import math
import numpy as np

def generate_spherocylinder_obj(filename, diameter=1, total_length=1.0, rings=16, segments=16, y_rotation_degrees=90):
    """
    Generate an OBJ file for a true spherocylinder (capsule) with rotation around the y-axis.
    A spherocylinder is a cylinder with hemispherical caps at both ends.
    
    Parameters:
    - filename: output OBJ file name
    - diameter: diameter of the capsule (default: 0.5)
    - total_length: total length of the capsule (default: 1.0)
    - rings: number of rings per hemisphere (default: 16)
    - segments: number of segments around the circumference (default: 16)
    - y_rotation_degrees: rotation angle in degrees around the y-axis (default: 90)
    """
    radius = diameter / 2.0
    cylinder_length = total_length - diameter  # Length of the cylindrical part
    
    if cylinder_length < 0:
        print("Warning: Total length is less than diameter. Creating a sphere instead.")
        cylinder_length = 0
    
    # Convert rotation angle to radians
    y_rotation = math.radians(y_rotation_degrees)
    
    vertices = []
    faces = []
    
    # Generate vertices
    
    # Bottom hemisphere cap (negative z)
    for i in range(rings//2 + 1):
        phi = -math.pi/2 + i * math.pi / rings
        for j in range(segments):
            theta = j * 2 * math.pi / segments
            x_orig = radius * math.cos(phi) * math.cos(theta)
            y_orig = radius * math.cos(phi) * math.sin(theta)
            z_orig = -cylinder_length/2 + radius * math.sin(phi)
            
            # Apply rotation around Y-axis: x' = x*cos(θ) + z*sin(θ), z' = -x*sin(θ) + z*cos(θ)
            x = x_orig * math.cos(y_rotation) + z_orig * math.sin(y_rotation)
            y = y_orig
            z = -x_orig * math.sin(y_rotation) + z_orig * math.cos(y_rotation)
            
            vertices.append((x, y, z))
    
    # Cylindrical middle
    for i in range(2):
        z_orig = (-0.5 + i) * cylinder_length
        for j in range(segments):
            theta = j * 2 * math.pi / segments
            x_orig = radius * math.cos(theta)
            y_orig = radius * math.sin(theta)
            
            # Apply rotation around Y-axis
            x = x_orig * math.cos(y_rotation) + z_orig * math.sin(y_rotation)
            y = y_orig
            z = -x_orig * math.sin(y_rotation) + z_orig * math.cos(y_rotation)
            
            vertices.append((x, y, z))
    
    # Top hemisphere cap (positive z)
    for i in range(rings//2 + 1):
        phi = i * math.pi / rings
        for j in range(segments):
            theta = j * 2 * math.pi / segments
            x_orig = radius * math.cos(phi) * math.cos(theta)
            y_orig = radius * math.cos(phi) * math.sin(theta)
            z_orig = cylinder_length/2 + radius * math.sin(phi)
            
            # Apply rotation around Y-axis
            x = x_orig * math.cos(y_rotation) + z_orig * math.sin(y_rotation)
            y = y_orig
            z = -x_orig * math.sin(y_rotation) + z_orig * math.cos(y_rotation)
            
            vertices.append((x, y, z))
    
    # Generate faces
    # For each section, we connect adjacent vertices to form triangular faces
    
    # Function to create a quad face (two triangles) from 4 vertex indices
    def add_quad(v1, v2, v3, v4):
        faces.append((v1+1, v2+1, v3+1))  # +1 because OBJ indices start at 1
        faces.append((v1+1, v3+1, v4+1))
    
    # Bottom hemisphere
    for i in range(rings//2):
        for j in range(segments):
            v1 = i * segments + j
            v2 = i * segments + (j+1) % segments
            v3 = (i+1) * segments + (j+1) % segments
            v4 = (i+1) * segments + j
            add_quad(v1, v2, v3, v4)
    
    # Connect bottom hemisphere to cylinder
    base_idx = (rings//2 + 1) * segments
    for j in range(segments):
        v1 = base_idx - segments + j
        v2 = base_idx - segments + (j+1) % segments
        v3 = base_idx + (j+1) % segments
        v4 = base_idx + j
        add_quad(v1, v2, v3, v4)
    
    # Cylinder part
    base_idx = (rings//2 + 1) * segments
    for j in range(segments):
        v1 = base_idx + j
        v2 = base_idx + (j+1) % segments
        v3 = base_idx + segments + (j+1) % segments
        v4 = base_idx + segments + j
        add_quad(v1, v2, v3, v4)
    
    # Connect cylinder to top hemisphere
    base_idx = (rings//2 + 1) * segments + segments
    for j in range(segments):
        v1 = base_idx + j
        v2 = base_idx + (j+1) % segments
        v3 = base_idx + segments + (j+1) % segments
        v4 = base_idx + segments + j
        add_quad(v1, v2, v3, v4)
    
    # Top hemisphere
    base_idx = (rings//2 + 1) * segments + 2 * segments
    for i in range(rings//2):
        for j in range(segments):
            v1 = base_idx + i * segments + j
            v2 = base_idx + i * segments + (j+1) % segments
            v3 = base_idx + (i+1) * segments + (j+1) % segments
            v4 = base_idx + (i+1) * segments + j
            add_quad(v1, v2, v3, v4)
    
    # Write to OBJ file
    with open(filename, 'w') as f:
        f.write("# Spherocylinder/Capsule OBJ file\n")
        f.write(f"# Diameter: {diameter}, Total Length: {total_length}\n")
        f.write(f"# Cylinder length: {cylinder_length}\n")
        f.write(f"# Y-axis rotation: {y_rotation_degrees} degrees\n")
        f.write(f"# Generated with {rings} rings and {segments} segments\n\n")
        
        for v in vertices:
            f.write(f"v {v[0]:.6f} {v[1]:.6f} {v[2]:.6f}\n")
        
        f.write("\n")
        for face in faces:
            f.write(f"f {face[0]} {face[1]} {face[2]}\n")
    
    print(f"Generated rotated spherocylinder OBJ file: {filename}")
    print(f"  - Vertices: {len(vertices)}")
    print(f"  - Faces: {len(faces)}")


if __name__ == "__main__":
    # Generate a spherocylinder with diameter 0.5 and total length 1.0, rotated 90 degrees around y-axis
    generate_spherocylinder_obj("spherocylinder.obj", diameter=0.5, total_length=1.0, y_rotation_degrees=90)

Generated rotated spherocylinder OBJ file: spherocylinder.obj
  - Vertices: 320
  - Faces: 608
